In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind

In [2]:
# VAMOS A VER ALGUNOS MÉTODOS DE VISUALIZACIÓN QUE NO CONOCIAMOS:

# establece el estilo del fondo de las gráficas como "whitegrid". Las gráficas tendrán un fondo blanco con líneas de cuadrícula horizontales y verticales.
sns.set_style("whitegrid")

# cambiará el esquema de colores predeterminado de Seaborn a uno más oscuro, adecuado para gráficas con fondos claros. 
sns.set_theme(style="dark")

# establece el estilo en Solarize_Lihgt2. 
plt.style.use('Solarize_Light2')

In [5]:
df = pd.read_csv("marketing_AB.csv", index_col = 0)
df.sample(10)

,user id,test group,converted,total ads,most ads day,most ads hour
96412,1308559,ad,False,20,Monday,17
151912,1487558,ad,False,13,Wednesday,17
227276,1584653,ad,False,24,Friday,15
477933,1198379,ad,False,2,Monday,16
14204,1542932,ad,False,66,Friday,19
83698,903346,psa,False,28,Sunday,13
467977,1306909,ad,False,14,Friday,10
372905,1482849,ad,False,17,Monday,22
75322,1499827,ad,False,54,Saturday,17
325752,1473194,ad,False,10,Wednesday,6


Ejercicio 1: Supongamos que estamos realizando un experimento para determinar si un nuevo diseño de anuncio (test group "ad") tiene un impacto significativo en la tasa de conversión en comparación con el diseño de anuncio anterior (test group "psa").

- Objetivo del ejercicio: Comparar las tasas de conversión entre los grupos de prueba "ad" y "psa" para determinar si el nuevo diseño de anuncio es más efectivo en la conversión de usuarios.

Pasos que debemos seguir:

Preparación de los datos: Asegúrate de que los datos estén limpios y listos para el análisis. Esto incluye la eliminación de datos faltantes y la verificación de la coherencia de los datos.

In [6]:
# una vez que tengamos nuestro DataFrame preparado con todas las columnas que queremos vamos a crear una función que no haga una exploración inicial del conjunto de datos
def exploracion_dataframe(df, columna_user):

    
    print(f"Los duplicados que tenemos en el conjunto de datos son: {df.duplicated().sum()}")
    print("\n ..................... \n")
    
    

    print("Los nulos que tenemos en el conjunto de datos son:")
    df_nulos = pd.DataFrame(df.isnull().sum() / df.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    print("\n ..................... \n")
    print(f"Los tipos de las columnas son:")
    display(pd.DataFrame(df.dtypes, columns = ["tipo_dato"]))
    
    
    print("\n ..................... \n")
    print("Los valores que tenemos para las columnas categóricas son: ")
    dataframe_categoricas = df.select_dtypes(include = "O")

    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valore únicos:")
        display(pd.DataFrame(df[col].value_counts()).head())    
    
    
    for categoria in df[columna_user].unique():
        
        dataframe_filtrado = df[df[columna_user] == categoria]
    
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas categóricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe(include = "O").T)
        
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas numéricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe().T)

In [7]:
exploracion_dataframe(df,"test group")

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos



 ..................... 

Los tipos de las columnas son:


,tipo_dato
user id,int64
test group,object
converted,bool
total ads,int64
most ads day,object
most ads hour,int64



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna TEST GROUP tiene las siguientes valore únicos:


...

,count,mean,std,min,25%,50%,75%,max
user id,564577.0,1.327314e+06,188918.061566,1000000.0,1163686.0,1327362.0,1490914.0,1654483.0
total ads,564577.0,2.482337e+01,43.750456,1.0,4.0,13.0,27.0,2065.0
most ads hour,564577.0,1.447590e+01,4.841808,0.0,11.0,14.0,18.0,23.0



 ..................... 

Los principales estadísticos de las columnas categóricas para el PSA son: 


,count,unique,top,freq
test group,23524,1,psa,23524
most ads day,23524,7,Thursday,3905



 ..................... 

Los principales estadísticos de las columnas numéricas para el PSA son: 


,count,mean,std,min,25%,50%,75%,max
user id,23524.0,911761.500000,6790.938202,900000.0,905880.75,911761.5,917642.25,923523.0
total ads,23524.0,24.761138,42.860720,1.0,4.00,12.0,26.00,907.0
most ads hour,23524.0,14.304923,4.656239,0.0,11.00,14.0,18.00,23.0


. Definición de las hipótesis: Plantea una hipótesis nula (H0) y una hipótesis alternativa (H1) para el experimento (en este caso os las damos definidas):

- Hipótesis nula (H0): No hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".
- Hipótesis alternativa (H1): Existe una diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

. Cálculo de la tasa de conversión: Calcula la tasa de conversión para cada grupo:

- Para el grupo "ad", la tasa de conversión es el número de usuarios convertidos dividido por el número total de usuarios en ese grupo.
- Para el grupo "psa", realiza el mismo cálculo.

In [8]:
df.sample(10)

,user id,test group,converted,total ads,most ads day,most ads hour
102069,1225754,ad,False,7,Tuesday,13
542354,1506485,ad,False,25,Monday,11
576626,1579496,ad,False,5,Sunday,20
573345,1361547,ad,False,2,Sunday,15
413671,1599739,ad,False,2,Wednesday,21
273484,1095019,ad,False,8,Wednesday,15
300423,909582,psa,False,19,Monday,20
177207,1080588,ad,False,90,Sunday,15
428484,1564067,ad,False,16,Wednesday,20
91892,1493513,ad,False,34,Friday,21


In [30]:
total_ad = df[df["test group"] == "ad"]["test group"].count()
total_ad

564577

In [31]:
tasa_conversion_ad = df['converted'] / total_ad
tasa_conversion_ad

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
588096    0.0
588097    0.0
588098    0.0
588099    0.0
588100    0.0
Name: converted, Length: 588101, dtype: float64

In [32]:
total_psa = df[df["test group"] == "psa"]["test group"].count()
total_psa

23524

In [33]:
tasa_conversion_psa = df['converted'] / total_psa
tasa_conversion_psa

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
588096    0.0
588097    0.0
588098    0.0
588099    0.0
588100    0.0
Name: converted, Length: 588101, dtype: float64

Prueba de hipótesis: Utiliza una prueba estadística, como la prueba t de Student o la prueba chi-cuadrado, para comparar las tasas de conversión entre los grupos "ad" y "psa".

In [36]:
def normalidad(df, columna):
    """
    Evalúa la normalidad de una columna de datos de un DataFrame utilizando la prueba de Shapiro-Wilk.

    Parámetros:
        dataframe (DataFrame): El DataFrame que contiene los datos.
        columna (str): El nombre de la columna en el DataFrame que se va a evaluar para la normalidad.

    Returns:
        None: Imprime un mensaje indicando si los datos siguen o no una distribución normal.
    """

    statistic, p_value = stats.shapiro(df[columna])
    if p_value > 0.05:
        print(f"Para la columna {columna} los datos siguen una distribución normal.")
    else:
        print(f"Para la columna {columna} los datos no siguen una distribución normal.")


In [ ]:
metricas = ['CTR', 'CR', 'CPA']

for metrica in metricas:
    normalidad(df ,metrica)

In [35]:
contingency = pd.crosstab(df['test group'], df['most ads hour'])
chi2, p_chi2, _, _ = chi2_contingency(contingency)
print(f"Prueba de Chi-cuadrado - p-valor: {p_chi2}")

Prueba de Chi-cuadrado - p-valor: 1.094573971081402e-28


In [ ]:
grupos = df[df['test group'] == 'ad']
test_group = df[df['test group'] == 'Test ad']
t_stat, p_ttest = ttest_ind(grupos['CTR'], test_group['CTR'])
print(f"Prueba t para CTR entre Control y Test - p-valor: {p_ttest}")